In [1]:
import os
import pandas as pd
import requests
import json
from pprint import pprint

In [4]:
# root folder
rootFolder = "bird_data"
# json downlad folder
jsonFolder = os.path.join(rootFolder,"json")
# audio folder
audioFolder = os.path.join(rootFolder,"audio")

In [5]:
birds_common_name = ["American Goldfinch", "American Robin", "Barn Swallow", "Blue Jay", "Blue-grey Gnatcatcher", 
                     "Broad-winged Hawk", "Carolina Chickadee", "Carolina Wren", "Cedar Waxwing", "Eastern Bluebird", 
                     "Northern Cardinal", "Rose-breasted Grosbeak","Ruby-crowned Kinglet", "Tufted Titmouse"]

birds_sci_name = ["Spinus tristis", "Turdus migratorius", "Hirundo rustica","Cyanocitta cristata", "Polioptila caerulea", 
                  "Buteo platypterus", "Poecile carolinensis", "Thryothorus ludovicianus", "Bombycilla cedrorum", "Sialia sialis", 
                  "Cardinalis cardinalis", "Pheucticus ludovicianus", "Regulus calendula","Baeolophus bicolor" ]

queryBirds = dict(zip(birds_common_name, birds_sci_name))

In [7]:
bird_df = pd.DataFrame(zip(birds_common_name, birds_sci_name), columns = ["common_name", "scientific_name"])
bird_df.to_csv(os.path.join(rootFolder,"birdList.csv"), index=False)

In [12]:
# function to build the query urls
def queryURL(queryBirds):
    endpoint = "https://www.xeno-canto.org/api/2/recordings?query="
    birdy = []
    for v in queryBirds.values():
        aux=v.split()
        birdy.append(aux)
    queryURL = []
    
    for i,j in birdy:
        url = f"{endpoint}+{i}+{j}+type:call+q_gt:D"
        queryURL.append(url)
        
    return queryURL
    
    

In [13]:
# function to request data by bird species
def requestBird(url):
    payload = {}
    response = requests.request("GET", url, data = payload).json()
    return response

In [14]:
# funcion to save json from the request
def jsonBird(response,bird, path):
    file_name = f'{bird}.json'
    file_path = os.path.join(path, file_name)
    with open(file_path,'w', encoding='utf-8') as f:
        json.dump(response,f, ensure_ascii=False, indent=4)
    f.close()
    

In [15]:
# request data
def xenoRequest(queryBirds, folder):
    bird_name = [k for k in queryBirds.keys()]
    birds = []
    for name in bird_name:
        bird_name = name.replace(" ","_")
        birds.append(bird_name)
    qURL = queryURL(queryBirds)
    for q,b in zip(qURL,birds):
        response = requestBird(q)
        jsonBird(response, b, folder)
    

In [16]:
xenoRequest(queryBirds, jsonFolder)